# LITERATURE IN THE AGE OF ARTIFICIAL INTELLIGENCE FINAL

1. All references listed in cells where they are used.

# Dependencies

In [3]:
# https://github.com/tensorflow/agents/blob/master/docs/tutorials/2_environments_tutorial.ipynb

!pip install tf-agents
!pip install 'gym==0.10.11'
!pip install fake-useragent
  
import abc
import tensorflow as tf
import numpy as np
import random
import time
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

tf.compat.v1.enable_v2_behavior()

     |████████████████████████████████| 911kB 3.4MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
  Found existing installation: gin-config 0.3.0
    Uninstalling gin-config-0.3.0:
      Successfully uninstalled gin-config-0.3.0
     |████████████████████████████████| 1.5MB 3.4MB/s 
  Created wheel for gym: filename=gym-0.10.11-cp36-none-any.whl size=1588313 sha256=5851c15440dae4c8f3934ae2ef828e6fc2d25b497ac9bc2785bd411a6b999cac
  Stored in directory: /root/.cache/pip/wheels/7b/eb/1f/22c4124f3c64943aa0646daf4612b1c1f00f27d89b81304ebd
Successfully built gym
  Found existing installation: gym 0.17.1
    Uninstalling gym-0.17.1:
      Successfully uninstalled gym-0.17.1
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13484 sha256=ee2f3d3cbe4479501ef4aa88a1070cfd22fcf12f2d2fb7956a6179446a4b586d
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built fake-useragent


# Environment Setup

In [0]:
'''
    # The poem as three arrays representing the following:
    # The poem consists of 12 sentences.
    
    first_word : a 1-d array where the index corresponds to the (sentence number - 1). The value of first_word[10] is 'Lest' from the 11th sentence.
    second_word : a 2-d array where the first index corresponds to the (sentence number - 1).
                                    the second index corresponds to one of (x-1) number of words that can be selected in that sentence.
                                    The value of second_word[10][4] is the 5th word (' Mind ') of the 11th sentence.
    third_word : a 1-d array where the index corresponds to the (sentence number - 1). The value of third_word[10] is 'Be Destroyed from the 11th sentence.

    note : Ending of 7th line has been changed from 'and g-d' to 'and god'
'''
first_word = ['from', 'come', 'from', 'come', 'the', 'love', 'the', 'seek', 'what', 'with', 'lest', 'where']

second_word = [
            [' night ', ' fog ', ' clash ', ' frost ', ' wind ', ' sea ', ' heat ', ' south ', ' east ', ' west ', ' north ', ' sun ', ' fire '],
            [' day ', ' blaze ', ' bloom ', ' snow ', ' peace ', ' land ', ' flash ', ' warmth ', ' heat ', ' joy ', ' cool ', ' light ', ' flames '],
            [' ache ', ' pain ', ' shame ', ' fear ', ' war ', ' oh ', ' cross ', ' stress ', ' scorn ', ' hurt ', ' shock ', ' guile ', ' scold '],
            [' joy ', ' charm ', ' fame ', ' ease ', ' wins ', ' truce ', ' gains ', ' peace ', ' jokes ', ' games ', ' rest ', ' mild '],

            [' moon ', ' beam ', ' smoke ', ' buck ', ' fish ', ' gold ', ' pearl ', ' tree ', ' flame ', ' stork ', ' frog ', ' lamb ox ', ' all '],
            [' sheen ', ' straw ', ' steam ', ' peak ', ' flood ', ' glow ', ' foam ', ' fruit ', ' ash ', ' roof ', ' pool ', ' field ', ' wheat '],
            [' guard ', ' man ', ' work ', ' strain ', ' art ', ' play ', ' ship ', ' mouth ', ' prince ', ' rage ', ' care ', ' greed ', ' faith '],
            [' goal ', ' sleep ', ' praise ', ' prize ', ' luck ', ' strife ', ' port ', ' kiss ', ' throne ', ' death ', ' shroud ', ' gold ', ' thanks '],

            [' good ', ' strong ', ' hard ', ' right ', ' long ', ' great ', ' white ', ' one ', ' yea ', ' air ', ' fire ', ' high wide ', ' should '],
            [' care ', ' bad ', ' weak ', ' plain ', ' bent ', ' broad ', ' small ', ' black ', ' three ', ' nay ', ' earth ', ' flood ', ' deep '],
            [' pluck ', ' love ', ' wise ', ' wit ', ' mind ', ' soul ', ' friend ', ' joy ', ' grace ', ' fame ', ' peace ', ' play ', ' praise '],
            [' fear ', ' hate ', ' lies ', ' wine ', ' meat ', ' flesh ', ' foe ', ' woe ', ' shame ', ' angst ', ' strife ', ' pain ', ' rule ']
         ]

third_word = ['and plagues', 'and dread', 'and fraud', 'morning rays', 'at play', 'and bread', 'and god', 'to say', 'be named', 'to avoid', 'be destroyed', 'the day']

In [5]:
# Quick Test to ensure embedding is loaded.

select = []

for x in range(len(first_word)):
  random.seed()

  selection = random.randint(0,len(second_word[x])-1)
  select.append(selection)

  search_term = first_word[x] + second_word[x][selection] + third_word[x]
  print(search_term)

print (select)

from wind and plagues
come snow and dread
from scold and fraud
come fame morning rays
the gold at play
love straw and bread
the prince and god
seek luck to say
what long be named
with weak to avoid
lest love be destroyed
where woe the day
[4, 3, 12, 2, 5, 1, 8, 4, 4, 2, 1, 7]


In [0]:
'''
USED TO RETURN GOOGLE SEARCH TOTALS
'''
def pyGoogleSearch(word,agent):
    header = {'User-Agent':str(agent.chrome)}
    address = 'http://www.google.com/search?q='
    newword = address + word
    page = requests.get(newword, headers=header)
    soup = BeautifulSoup(page.content, 'html.parser')
    res = soup.find("div", {"id": "result-stats"})
    try:
      final = res.text.replace(",", "").split()[1]
      final = int(final)
      return final
    except:
      return 0

In [0]:
'''
TEST AREA PRIOR TO DEVELOPMENT OF REINFORCEMENT LEARNING ENVIRONMENT
'''
ua = UserAgent()

# 
sec_word_array = []
scores = []
reward = 0

for x in range(len(first_word)):
  random.seed()

  # Load search term
  selection = random.randint(0,len(second_word[x])-1)
  sec_word_array.append(selection)
  search_term = "\"" +  first_word[x] + second_word[x][selection] + third_word[x] + "\""

  # return results for searching line
  score = pyGoogleSearch(search_term, ua)
  scores.append(score)
  
  # add total number of results
  reward += score
  print("Search Term: %s -- Results: %d" %(search_term, score))

print( sec_word_array )
print( scores )
print( reward )

In [0]:
'''
INITIAL REINFORCEMENT LEARNING ENVIRONMENT
'''

class Kuhlmann(py_environment.PyEnvironment):

  def __init__(self):
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int32, minimum=0, maximum=1, name='action')
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(2,10), dtype=np.int32, minimum=0, name='observation')
    self._state = array_spec.BoundedArraySpec(
        shape=(2,10), dtype=np.int32, minimum=0, name='state')
    self._episode_ended = False
    self.ua = UserAgent()
  
  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def _reset(self):
    self._state = 0
    self._episode_ended = False
    return ts.restart(np.array([self._state], dtype=np.int32))

  def _step(self, action):
    if self._episode_ended:
      # The last action ended the episode. Ignore the current action and start
      # a new episode.
      return self.reset()

    # Make sure episodes don't go on forever.
    if action == 1:
      self._episode_ended = True
    elif action == 0:
      #generate poem
      sec_word_array = []
      scores = []
      total_reward = 0

      for x in range(len(first_word)):
        random.seed()

        # Load search term
        selection = random.randint(0,len(second_word[x])-1)
        sec_word_array.append(selection)
        search_term = "\"" +  first_word[x] + second_word[x][selection] + third_word[x] + "\""

        # return results for searching line
        score = pyGoogleSearch(search_term, self.ua)
        scores.append(score)
        
        # add total number of results
        total_reward += score
      
      self._state = [sec_word_array, scores]
    else:
      raise ValueError('`action` should be 0 or 1.')

    if self._episode_ended:
      reward = total_reward
      return ts.termination(np.array([self._state], dtype=np.int32), reward)
    else:
      return ts.transition(
          np.array([self._state], dtype=np.int32), reward=0.0, discount=1.0)

In [0]:
'''
FOR TESTING PURPOSES
'''
environment = Kuhlmann()
utils.validate_py_environment(environment, episodes=5)

In [0]:
'''
FOR TESTING PURPOSES
'''

generate_poem = np.array(0, dtype=np.int32)
record_stats = np.array(1, dtype=np.int32)

environment = Kuhlmann()
time_step = environment.reset()
print(time_step)
cumulative_reward = time_step.reward

for _ in range(5):
  time_step = environment.step(generate_poem)
  print(time_step)
  cumulative_reward += time_step.reward

time_step = environment.step(record_stats)
print(time_step)
cumulative_reward += time_step.reward
print('Final Reward = ', cumulative_reward)

TimeStep(step_type=array(0, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([0], dtype=int32))
TimeStep(step_type=array(1, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([9], dtype=int32))
TimeStep(step_type=array(1, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([11], dtype=int32))
TimeStep(step_type=array(1, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([15], dtype=int32))
TimeStep(step_type=array(1, dtype=int32), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([17], dtype=int32))
TimeStep(step_type=array(2, dtype=int32), reward=array(-21., dtype=float32), discount=array(0., dtype=float32), observation=array([26], dtype=int32))
TimeStep(step_type=array(2, dtype=int32), reward=array(-21., dtype=float32), discount=array(0., dtype=float32), 

# MANUAL TESTING

This section was created after manually obtaining total search results.


In [0]:
# BELOW ARE ALL PHRASES SEARCHED AND TOTAL SEARCH RESULTS

'''
"from night and plagues"
"from fog and plagues"
"from clash and plagues"
"from frost and plagues" 1
"from wind and plagues"
"from sea and plagues"
"from heat and plagues"
"from south and plagues"
"from east and plagues"
"from west and plagues"
"from north and plagues"
"from sun and plagues"
"from fire and plagues" 10
[0,0,0,1,0,0,0,0,0,0,0,0,10]

"come day and dread"
"come blaze and dread"
"come bloom and dread"
"come snow and dread"
"come peace and dread"
"come land and dread"
"come flash and dread"
"come warmth and dread"
"come heat and dread"
"come joy and dread"
"come cool and dread"
"come light and dread"
"come flames and dread"
[0,0,0,0,0,0,0,0,0,0,0,0,0]

"from ache and fraud"
"from pain and fraud" 1
"from shame and fraud"
"from fear and fraud" 24500
"from war and fraud" 5
"from oh and fraud" 5
"from cross and fraud"
"from stress and fraud" 1
"from scorn and fraud"
"from hurt and fraud"
"from shock and fraud"
"from guile and fraud" 4
"from scold and fraud"
[0,1,0,24500,5,5,0,1,0,0,0,4,0]

"come joy morning rays"
"come charm morning rays"
"come fame morning rays"
"come ease morning rays"
"come wins morning rays"
"come truce morning rays"
"come gains morning rays"
"come peace morning rays"
"come jokes morning rays"
"come games morning rays"
"come rest morning rays"
"come mild morning rays"
[0,0,0,0,0,0,0,0,0,0,0,0,0]

"the moon at play" 27800
"the beam at play"
"the smoke at play" 1
"the buck at play" 1
"the fish at play" 22400
"the gold at play"
"the pearl at play" 2
"the tree at play" 38100
"the flame at play" 1
"the stork at play" 6
"the frog at play" 6
"the lamb ox at play"
"the all at play" 3
[27800,0,1,1,22400,0,2,38100,1,6,6,0,3]


"love sheen and bread"
"love straw and bread"
"love steam and bread"
"love peak and bread"
"love flood and bread"
"love glow and bread"
"love foam and bread"
"love fruit and bread" 15900
"love ash and bread"
"love roof and bread"
"love pool and bread"
"love field and bread"
"love wheat and bread" 5
[0,0,0,0,0,0,0,15900,0,0,0,0,5]


"the guard and god" 121000
"the man and god" 6710000
"the work and god" 6840000
"the strain and god" 56300
"the art and god" 2080000
"the play and god" 289000
"the ship and god" 308000
"the mouth and god" 2
"the prince and god" 278000
"the rage and god" 86000
"the care and god" 7
"the greed and god" 46100
"the faith and god" 6290000
[121000,6710000,6840000,56300,2080000,289000,308000,2,278000,86000,7,46100,6290000]


"seek goal to say"
"seek sleep to say"
"seek praise to say" 1
"seek prize to say"
"seek luck to say"
"seek strife to say"
"seek port to say"
"seek kiss to say"
"seek throne to say"
"seek death to say"
"seek shroud to say"
"seek gold to say"
"seek thanks to say"
[0,0,1,0,0,0,0,0,0,0,0,0,0]


"what good be named"
"what strong be named"
"what hard be named"
"what right be named"
"what long be named"
"what great be named"
"what white be named"
"what one be named"
"what yea be named"
"what air be named"
"what fire be named"
"what high wide be named"
"what should be named" 1550000
[0,0,0,0,0,0,0,0,0,0,0,0,1550000]


"with care to avoid" 4980000
"with bad to avoid" 4
"with weak to avoid"
"with plain to avoid" 19000
"with bent to avoid"
"with broad to avoid" 1
"with small to avoid"
"with black to avoid" 389000
"with three to avoid" 4
"with nay to avoid" 1
"with earth to avoid" 194000
"with flood to avoid" 1
"with deep to avoid" 4
[4980000,4,0,19000,0,1,0,389000,4,1,194000,1,4]


"lest pluck be destroyed"
"lest love be destroyed"
"lest wise be destroyed"
"lest wit be destroyed"
"lest mind be destroyed"
"lest soul be destroyed"
"lest friend be destroyed"
"lest joy be destroyed"
"lest grace be destroyed"
"lest fame be destroyed"
"lest peace be destroyed" 3
"lest play be destroyed"
"lest praise be destroyed"
[0,0,0,0,0,0,0,0,0,0,3,0,0]


"where fear the day" 1
"where hate the day"
"where lies the day"
"where wine the day"
"where meat the day"
"where flesh the day"
"where foe the day" 1
"where woe the day" 1
"where shame the day"
"where angst the day"
"where strife the day"
"where pain the day"
"where rule the day"
[1,0,0,0,0,0,1,1,0,0,0,0,0]
'''

In [6]:
# All results from the above cell as one matrix.

google_results = [
                  [0,0,0,1,0,0,0,0,0,0,0,0,10],
                  [0,0,0,0,0,0,0,0,0,0,0,0,0],
                  [0,1,0,24500,5,5,0,1,0,0,0,4,0],
                  [0,0,0,0,0,0,0,0,0,0,0,0,0],
                  [27800,0,1,1,22400,0,2,38100,1,6,6,0,3],
                  [0,0,0,0,0,0,0,15900,0,0,0,0,5],
                  [121000,6710000,6840000,56300,2080000,289000,308000,2,278000,86000,7,46100,6290000],
                  [0,0,1,0,0,0,0,0,0,0,0,0,0],
                  [0,0,0,0,0,0,0,0,0,0,0,0,1550000],
                  [4980000,4,0,19000,0,1,0,389000,4,1,194000,1,4],
                  [0,0,0,0,0,0,0,0,0,0,3,0,0],
                  [1,0,0,0,0,0,1,1,0,0,0,0,0]
                  ]
np_arr = np.array(google_results)
np_arr.shape

(12, 13)

In [0]:
def reconstruct(second_word_array):
  for index, val in enumerate(second_word_array):
    if index % 4 == 0:
      print(" ")
    print(first_word[index]+second_word[index][val]+third_word[index])
  
  print (" ")
  print ("all things do change; all things do love; all things are locked in hate")
  if np.sum(second_word_array) % 2 == 0:
    print('whoever fully thinks this through holds the key to man\'s estate')
  else:
    print('holds the key to man\'s estate whoever fully thinks this through')


In [10]:
'''
  SET RESULTS THRESHOLD BY ALTERING FIRST VARIABLE 'min_results_threshold' BELOW.
'''

min_results_threshold = 5000

max_val_output = []
np_gr = np.array(google_results)
for row in np_gr:
  random.seed()
  selection = random.randint(0,len(second_word[x])-1)
  if row.sum() > min_results_threshold:
    max_val_output.append(row.argmax(axis=0))
  else:
    max_val_output.append(selection)

reconstruct(max_val_output)
print(max_val_output)

 
from east and plagues
come bloom and dread
from fear and fraud
come wins morning rays
 
the tree at play
love fruit and bread
the work and god
seek strife to say
 
what should be named
with care to avoid
lest mind be destroyed
where fear the day
 
all things do change; all things do love; all things are locked in hate
whoever fully thinks this through holds the key to man's estate
[8, 2, 3, 4, 7, 7, 2, 5, 12, 0, 4, 0]
